In [16]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import train_test_split
import cv2

In [2]:
data_dir = 'drive/MyDrive/kaggle_mnist/digit-recognizer/'

for dirname, _, filenames in os.walk('drive/MyDrive/kaggle_mnist/digit-recognizer'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

drive/MyDrive/kaggle_mnist/digit-recognizer/test.csv
drive/MyDrive/kaggle_mnist/digit-recognizer/sample_submission.csv
drive/MyDrive/kaggle_mnist/digit-recognizer/train.csv


In [3]:
train_data = pd.read_csv(data_dir + 'train.csv')
test_data = pd.read_csv(data_dir + 'test.csv')
submssion = pd.read_csv(data_dir + 'sample_submission.csv')

In [21]:
train_images = train_data.drop('label', axis=1).values.reshape(-1, 1, 28, 28)
test_images = test_data.values.reshape(-1, 1, 28, 28)
labels = train_data['label'].values

# nomalization
train__images = train_images / 255.0
test_images = test_images / 255.0

print(train_images.shape)
print(test_images.shape)

(42000, 1, 28, 28)
(28000, 1, 28, 28)


In [22]:
train_images, valid_images, train_labels, valid_labels = train_test_split(train_images, labels, test_size=0.2, random_state=42, stratify=labels)

transform = transforms.Compose([
    transforms.RandomRotation(degrees=30),
    transforms.ToTensor()
])

train_tensor = torch.utils.data.TensorDataset(torch.tensor(train_images), torch.tensor(train_labels))
val_tensor = torch.utils.data.TensorDataset(torch.tensor(valid_images), torch.tensor(valid_labels))
test_tensor = torch.utils.data.TensorDataset(torch.tensor(test_images))

train_loader = DataLoader(train_tensor, batch_size=32, shuffle=True)
val_loader = DataLoader(val_tensor, batch_size=32, shuffle=False)
test_loader = DataLoader(test_tensor, batch_size=32, shuffle=False)

In [31]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3),
            nn.ReLU()
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=16, kernel_size=3),
            nn.Conv2d(in_channels=16, out_channels=1, kernel_size=3),
            nn.Softmax()
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        return x

In [32]:
model = Network()

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)

Network(
  (layer1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
    (2): ReLU()
  )
  (layer2): Sequential(
    (0): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): Conv2d(16, 1, kernel_size=(3, 3), stride=(1, 1))
    (2): Softmax(dim=None)
  )
)

In [33]:
# loss function
critreion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
num_epochs = 100

for epoch in range(1, num_epochs + 1):

    # train
    model.train()
    for step in range(len(train_loader)):
        images, labels = iter(train_loader)
        predictions = model(images)
        optimizer.zero_grad()
        loss = criterion(predictions, labels)
        loss.backward()
        optimizer.step()

    # validation
    model.eval()
    with torch.no_grad():
        for step in range(len(valid_loader)):
            images, labels = iter(valid_loader)
            predictions = model(images)
            loss = criterion(predictions, labels)
